# Czym jest SPARQL

**SPARQL** (SPARQL Protocol And RDF Query Language):
- język zapytań do danych w formacie RDF
- protokół (REST)

W celu ćwiczeń użyjemy przykładowego grafu opisującego domenę miejscowości, który zawiera wybrane informacje z grafu wiedzy **DBpedia** (https://www.dbpedia.org). 
DBpedia zawiera dane z Wikipedii ustrukturyzowane do formatu RDF.

In [1]:
!pip install rdflib

from rdflib import Graph
g = Graph()

g.parse("miejscowosci3.ttl", format="ttl")
print("Graf zawiera %s trójek." % len(g))

Graf zawiera 156 trójek.


W rdflib zapytania SPARQL mogą być wydawane do grafu za pomocą metody rdflib.graph.Graph.query().

# Wzorce trójkowe i podstawowy wzorzec
Główną formą zapytania w SPARQL jest zapytanie typu `SELECT`, które wygląda trochę jak zapytanie SQL. Zapytanie `SELECT` składa się z dwóch głównych elementów: nagłówka z listą wybranych zmiennych i klauzuli `WHERE` do określenia wzorców grafów jakie chcemy dopasować do zapytania, a konkretniej **podstawowego wzorca grafu** (zapisanego w nawiasach klamrowych). 

Wynikiem zapytania `SELECT` jest tabela, w której będzie jedna kolumna dla każdej wybranej zmiennej i jeden wiersz dla każdego dopasowania do wzorca.

Podstawowym elementem składowym zapytań SPARQL są **wzorce trójkowe**. Są one podobne do trójek RDF, ale możesz użyć zmiennej w dowolnej z trzech pozycji. Używamy ich aby znaleźć pasujące trójki w grafie, a zmienne działają jak symbole wieloznaczne pasujące do dowolnego węzła grafu.

In [2]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       
       SELECT ?x ?y
       WHERE {
          ?x dbo:country ?y .
       }""")

for row in qres:
    print("%s należy do kraju %s" % row)

http://dbpedia.org/resource/Poznań należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Wrocław należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Malbork należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Ogrodzieniec należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Uchylsko należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Tricity,_Poland należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Kraków należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Gliwice należy do kraju http://dbpedia.org/resource/Poland
http://dbpedia.org/resource/Warsaw należy do kraju http://dbpedia.org/resource/Poland


<span style="color:red"> __Zadanie 1: Sformułuj proste zapytanie do grafu g (zawierające jeden wzorzec trójkowy) o encje dotyczące obiektów mających swoje lokalizacje (`dbo:location`) w Warszawie. Warszawa reprezentowana jest poprzez zasób `dbr:Warsaw`, gdzie `dbr` to prefiks związany z przestrzenią nazw <http://dbpedia.org/resource/>). Na liście wyników powinien znaleźć się m.in. zasób http://dbpedia.org/resource/Copernicus_Science_Centre.    __ </span>

In [15]:
qres = g.query("""
    PREFIX dbo: <http://dbpedia.org/ontology/> 
    PREFIX dbr: <http://dbpedia.org/resource/> 
    SELECT ?x
    WHERE {?x dbo:location dbr:Warsaw}
""")
for row in qres:
    print("%s ma lokalizację w Warszawie" % row)

http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw ma lokalizację w Warszawie
http://dbpedia.org/resource/Konstanty_Zamoyski_Palace ma lokalizację w Warszawie
http://dbpedia.org/resource/Miodowa_Street_(Warsaw) ma lokalizację w Warszawie
http://dbpedia.org/resource/Copernicus_Science_Centre ma lokalizację w Warszawie
http://dbpedia.org/resource/Grochowska_Street,_Warsaw ma lokalizację w Warszawie
http://dbpedia.org/resource/Uruski_Palace ma lokalizację w Warszawie


Zadajmy następnie zapytanie zawierające dwa wzorce trójek, o obiekty geograficzne, które znajdują się w konkretnych dzielnicach Warszawy:

In [16]:
  qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       
       select ?poi ?district
       WHERE {
        dbr:Warsaw dbo:subdivision ?district   .
        ?poi dbo:location ?district .
       }
       """)

for row in qres:
    print("%s leży w %s" % row)

http://dbpedia.org/resource/Warsaw_Uprising_Museum leży w http://dbpedia.org/resource/Wola
http://dbpedia.org/resource/Electio_Viritim_Monument leży w http://dbpedia.org/resource/Wola
http://dbpedia.org/resource/Korkowa_Street,_Warsaw leży w http://dbpedia.org/resource/Wawer
http://dbpedia.org/resource/Grochowska_Street,_Warsaw leży w http://dbpedia.org/resource/Praga_Południe
http://dbpedia.org/resource/Arena_COS_Torwar leży w http://dbpedia.org/resource/Śródmieście,_Warsaw
http://dbpedia.org/resource/Chancellery_of_the_Prime_Minister_of_Poland leży w http://dbpedia.org/resource/Śródmieście,_Warsaw
http://dbpedia.org/resource/Castle_Square,_Warsaw leży w http://dbpedia.org/resource/Śródmieście,_Warsaw
http://dbpedia.org/resource/Warsaw_Icon_Museum leży w http://dbpedia.org/resource/Ochota
http://dbpedia.org/resource/Museum_of_John_Paul_II_and_Primate_Wyszyński leży w http://dbpedia.org/resource/Wilanów
http://dbpedia.org/resource/Warsaw_Chopin_Airport leży w http://dbpedia.org/resourc

<span style="color:red"> __Zadanie 2: Sformułuj do grafu g zapytanie (zawierające dwa wzorce trójkowe) o typy obiektów geograficznych, które znajdują się w Warszawie (wykorzystaj własności `dbo:location` i `rdf:type`). Na liście wyników powinna znaleźć się m.in. para: (http://dbpedia.org/resource/Copernicus_Science_Centre, http://dbpedia.org/ontology/Museum) __ </span>

In [18]:
qres = g.query("""
    PREFIX dbo: <http://dbpedia.org/ontology/> 
    PREFIX dbr: <http://dbpedia.org/resource/> 
    SELECT ?x ?y
    WHERE {
        ?x dbo:location dbr:Warsaw .
        ?x rdf:type ?y .
    }
""")

for row in qres:
    print("%s jest typu %s" % row)

http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw jest typu http://dbpedia.org/ontology/Place
http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw jest typu http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw jest typu http://www.w3.org/2002/07/owl#NamedIndividual
http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw jest typu http://dbpedia.org/ontology/Building
http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw jest typu http://dbpedia.org/ontology/ArchitecturalStructure
http://dbpedia.org/resource/Holy_Trinity_Church,_Warsaw jest typu http://dbpedia.org/ontology/Location
http://dbpedia.org/resource/Konstanty_Zamoyski_Palace jest typu http://dbpedia.org/ontology/Place
http://dbpedia.org/resource/Konstanty_Zamoyski_Palace jest typu http://www.w3.org/2002/07/owl#Thing
http://dbpedia.org/resource/Konstanty_Zamoyski_Palace jest typu http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
http://dbpedia.org/resource/Kon

# Modyfikatory zapytania

Jeżeli chcielibyśmy żeby wyniki zapytania były posortowane wg wartości wybranej zmiennej, możemy dodać klazulę `ORDER BY`:

In [19]:
  qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       
       select ?poi ?district 
       WHERE {
        dbr:Warsaw dbo:subdivision ?district   .
        ?poi dbo:location ?district .
       }
       ORDER BY ?poi
       """)

for row in qres:
    print("%s leży w %s" % row)

http://dbpedia.org/resource/Arena_COS_Torwar leży w http://dbpedia.org/resource/Śródmieście,_Warsaw
http://dbpedia.org/resource/Castle_Square,_Warsaw leży w http://dbpedia.org/resource/Śródmieście,_Warsaw
http://dbpedia.org/resource/Chancellery_of_the_Prime_Minister_of_Poland leży w http://dbpedia.org/resource/Śródmieście,_Warsaw
http://dbpedia.org/resource/Electio_Viritim_Monument leży w http://dbpedia.org/resource/Wola
http://dbpedia.org/resource/Grochowska_Street,_Warsaw leży w http://dbpedia.org/resource/Praga_Południe
http://dbpedia.org/resource/Korkowa_Street,_Warsaw leży w http://dbpedia.org/resource/Wawer
http://dbpedia.org/resource/Museum_of_John_Paul_II_and_Primate_Wyszyński leży w http://dbpedia.org/resource/Wilanów
http://dbpedia.org/resource/Warsaw_Chopin_Airport leży w http://dbpedia.org/resource/Włochy
http://dbpedia.org/resource/Warsaw_Icon_Museum leży w http://dbpedia.org/resource/Ochota
http://dbpedia.org/resource/Warsaw_Uprising_Museum leży w http://dbpedia.org/resou

Modyfikator `LIMIT` pozwala nam z kolei na wyświetlenie ograniczonej liczby wyników:

In [20]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       
       select ?poi ?district 
       WHERE {
        dbr:Warsaw dbo:subdivision ?district   .
        ?poi dbo:location ?district .
       }
       LIMIT 3
       """)

for row in qres:
    print("%s leży w %s" % row)

http://dbpedia.org/resource/Warsaw_Uprising_Museum leży w http://dbpedia.org/resource/Wola
http://dbpedia.org/resource/Electio_Viritim_Monument leży w http://dbpedia.org/resource/Wola
http://dbpedia.org/resource/Korkowa_Street,_Warsaw leży w http://dbpedia.org/resource/Wawer


 <span style="color:red"> __Zadanie 3: sformułuj do grafu g zapytanie o miasta leżące w Polsce ograniczając wyniki do 5  __ </span>

In [39]:
qres = g.query("""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    select ?x
    WHERE {
        ?x dbo:country dbr:Poland .
    }
    LIMIT 5
""")

for row in qres:
    print("%s" % row)

http://dbpedia.org/resource/Wrocław
http://dbpedia.org/resource/Poznań
http://dbpedia.org/resource/Kraków
http://dbpedia.org/resource/Ogrodzieniec
http://dbpedia.org/resource/Uchylsko


# Klauzula FILTER



Klauzula FILTER pozwala na wykluczenie wybranych trójek z wyników zapytania. Jej ideą jest wykonanie boolowskiego testu, który ma na celu włączenie bądź też wykluczenie wyników na podstawie wartości danej zmiennej. 

SPARQL obsługuje wiele wbudowanych funkcji do pisania takich wyrażeń, np.:
operatory porównania: (`=`,`!=`, `<`, `<=`, `>`, `>=`)
operatory logiczne (`&&`, `||`, `!`)
operatory matematyczne (`+`, `-`, `/`, `*`)  


In [40]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>

       SELECT ?settlement ?population
       WHERE {
        ?settlement dbo:country dbr:Poland .
        ?settlement dbo:populationTotal ?population . FILTER (?population >= "500000"^^xsd:int)
       }""")

for row in qres:
    print("%s ma liczbę ludności %s" % row)

http://dbpedia.org/resource/Wrocław ma liczbę ludności 642869
http://dbpedia.org/resource/Poznań ma liczbę ludności 534813
http://dbpedia.org/resource/Kraków ma liczbę ludności 779115
http://dbpedia.org/resource/Tricity,_Poland ma liczbę ludności 748986
http://dbpedia.org/resource/Warsaw ma liczbę ludności 1790658


<span style="color:red"> __Zadanie 4: sformułuj do grafu g zapytanie o nazwy miejscowości i ich powierznie (`dbo:areaTotal`), które mają powierzchnie większe niż 120000000 __ </span>

In [56]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

       SELECT ?city ?area
       WHERE {
        ?city dbo:country dbr:Poland .
        ?city dbo:areaTotal ?area . FILTER (?area >= "120000000"^^xsd:int)
       }""")

for row in qres:
    print("%s\tma liczbę ludności %s" % row)

http://dbpedia.org/resource/Wrocław	ma liczbę ludności 292920000.0
http://dbpedia.org/resource/Poznań	ma liczbę ludności 261850000.0
http://dbpedia.org/resource/Kraków	ma liczbę ludności 326800000.0
http://dbpedia.org/resource/Tricity,_Poland	ma liczbę ludności 414810000.0
http://dbpedia.org/resource/Gliwice	ma liczbę ludności 133880000.0
http://dbpedia.org/resource/Warsaw	ma liczbę ludności 517240000.0


# Klauzula OPTIONAL

Za pomocą klauzuli OPTIONAL możemy wyspecyfikować fragmenty zapytania, które nie muszą być dopasowane do grafu aby całe zapytanie zwróciło wynik dla danego wzorca grafu. Przykładowo, w grafie wiedzy takim jak DBpedia mogą znajdować się informacje na temat liczby ludności danej miejscowości, ale nie na temat jej powierzchni, mimo tego możemy chcieć zwrócić informacje o danej miejscowości, nawet jeśli są cząstkowe.

In [57]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>

       SELECT ?settlement ?population
       WHERE {
        ?settlement dbo:country dbr:Poland .
        OPTIONAL {?settlement dbo:populationTotal ?population . }
       }""")

for row in qres:
    print("%s ma liczbę ludności %s" % row)


http://dbpedia.org/resource/Wrocław ma liczbę ludności 642869
http://dbpedia.org/resource/Poznań ma liczbę ludności 534813
http://dbpedia.org/resource/Kraków ma liczbę ludności 779115
http://dbpedia.org/resource/Ogrodzieniec ma liczbę ludności 4282
http://dbpedia.org/resource/Uchylsko ma liczbę ludności 338
http://dbpedia.org/resource/Tricity,_Poland ma liczbę ludności 748986
http://dbpedia.org/resource/Malbork ma liczbę ludności 38723
http://dbpedia.org/resource/Gliwice ma liczbę ludności 178603
http://dbpedia.org/resource/Warsaw ma liczbę ludności 1790658


<span style="color:red"> __Zadanie 5: Sformułuj zapytanie o nazwy miejscowości z opcjonalną informacją o jej powierzchni (`dbo:areaTotal`). __ </span>

In [58]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

       SELECT ?city ?area
       WHERE {
        ?city dbo:country dbr:Poland .
        OPTIONAL {?city dbo:areaTotal ?area}
       }""")

for row in qres:
    print("%s\tma liczbę ludności %s" % row)

http://dbpedia.org/resource/Wrocław	ma liczbę ludności 292920000.0
http://dbpedia.org/resource/Poznań	ma liczbę ludności 261850000.0
http://dbpedia.org/resource/Kraków	ma liczbę ludności 326800000.0
http://dbpedia.org/resource/Ogrodzieniec	ma liczbę ludności None
http://dbpedia.org/resource/Uchylsko	ma liczbę ludności 2520000.0
http://dbpedia.org/resource/Tricity,_Poland	ma liczbę ludności 414810000.0
http://dbpedia.org/resource/Malbork	ma liczbę ludności None
http://dbpedia.org/resource/Gliwice	ma liczbę ludności 133880000.0
http://dbpedia.org/resource/Warsaw	ma liczbę ludności 517240000.0


# Zapytanie typu ASK

Jeśli zależy nam na określeniu czy dany wzorzec trójkowy albo trójka RDF w ogóle znajdzie dopasowanie w grafie a niekoniecznie na wszystkich wynikach dopasowania, możemy zadać zapytanie typu `ASK`, które zwraca wartość `true` albo `false`: 

In [60]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       ASK 
       WHERE {
          dbr:Warsaw dbo:location dbr:Poland 
       }""")

for row in qres:
    print("%s" % row)

False


<span style="color:red"> __Zadanie 6: Sformułuj zapytanie typu `ASK` analogiczne do powyższego, ale o to czy krajem Warszawy jest Polska (`dbo:country`). __ </span>

In [61]:
qres = g.query(
    """PREFIX dbo: <http://dbpedia.org/ontology/>
       PREFIX dbr: <http://dbpedia.org/resource/>
       ASK 
       WHERE {
          dbr:Warsaw dbo:country dbr:Poland 
       }""")

for row in qres:
    print("%s" % row)

True


# Końcówka SPARQL DBpedii

Zapytania SPARQL są wykonywane względem zbiorów danych RDF, składających się z grafów RDF. 
Końcówka SPARQL to usługa, która akceptuje zapytania i zwraca wyniki przez HTTP. 
Końcówki SPARQL mają swoje adresy, najczęściej powiązane z konkretnymi zbiorami danych. 
Adres końcówki SPARQL powiązanej z grafem wiedzy DBpedia to https://dbpedia.org/sparql
DBpedia oferuje także interfejsy służące do przeglądania grafu jak i do jego odpytywania: https://dbpedia.org/sparql/
 

__Zadanie 7: Korzystając z interfejsu jaki udostępnia DBpedia i pozyskanej do tej pory wiedzy na temat tego grafu, sformułuj następujące zapytania w języku SPARQL, tak żeby dostać wyniki za pomocą końcówki SPARQL DBpedii:

1. Lista osób urodzonych w Warszawie 
2. Lista muzeów w Krakowie
3. Lista osób urodzonych w Warszawie, które zdobyły nagrodę Nobla
4. Daty urodzin osób urodzonych w Krakowie  
. __

In [62]:
# prefiksy takie same jak w poprzednich zadaniach

urodzeni_warszawa = """
SELECT ?x
WHERE {
  ?x rdf:type dbo:Person .
  ?x dbo:birthPlace dbr:Warsaw .
}
"""

muzea_kraków = """
SELECT ?x
WHERE {
  ?x rdf:type dbo:Museum .
  ?x dbo:location dbr:Kraków . 
}
"""

# http://data.nobelprize.org/snorql/
nobliści_warszawa = """
SELECT ?label
WHERE { 
  ?x rdf:type <http://data.nobelprize.org/terms/Laureate> . 
  ?x rdfs:label ?label .
  ?x dbpedia-owl:birthPlace <http://data.nobelprize.org/resource/city/Warsaw> .
}
"""

urodziny_warszawa = """
SELECT ?x ?birthday
WHERE {
  ?x rdf:type dbo:Person .
  ?x dbo:birthDate ?birthday .
  ?x dbo:birthPlace dbr:Warsaw .
}
"""